In [47]:
%matplotlib qt5

In [1]:
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
from mathphys.functions import save_pickle, load_pickle

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass
import pyaccel.optics as pyop

In [89]:
def track_mchn_stdy(acc, bunch, param, increment, index, nturn):
    # setting param to be the index of the array [x,x',y,y',delta,z]

    bunch[param] += increment
    tracked = pyaccel.tracking.ring_pass(acc, particles=bunch,
                                         nr_turns=nturn, turn_by_turn= True,
                                         element_offset= index, parallel=True)
    
    part_out, lost_flag, turn_lost, index_lost, plane_lost = tracked

    turnl_element = []

    for i,iten in enumerate(turn_lost):
        if iten == nturn and index_lost[i] == index: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((iten, index_lost[i]))
    
    return turnl_element

def varying_incmnts(acc,bunch,par,rng, index, nturn, val):
    trackf_lostpos = []
    inc = np.linspace(0, val, rng)
    incdif = np.diff(inc)
    for iten in incdif:
        
        trackf_lostpos.append(track_mchn_stdy(acc, bunch, par, increment=iten, index=index, nturn=nturn))

    return trackf_lostpos

def plot_tracked(acc,l_of_tples, idx, spos, val): 
    
    inc = np.linspace(0,val, len(l_of_tples))
    for c, iten in enumerate(l_of_tples):
        if len(iten) == 0:
            pass
        else:
            j = c
            break

    if len(l_of_tples[idx]) == 0:
        print("all the electrons survived")
        return

    else:
        nt_lost, index = np.zeros(len(l_of_tples[idx])), np.zeros(len(l_of_tples[idx]))

        for j,iten in enumerate(l_of_tples[idx]):
            nt_lostj, idx_lost = iten
            nt_lost[j] = nt_lostj
            index[j] = idx_lost
        index = np.intp(index)

    ind = np.arange(index.size)

    plt.figure()
    plt.title('electron losses by x parameter variation', fontsize=16) # . incremento {:.3f}'.format(inc[idx])
    plt.scatter(spos[index],ind, s=4, label='perde a partir de {}'.format(inc[c]))
    pyaccel.graphics.draw_lattice(acc, offset=0, height=3, gca=True)
    plt.legend()

    plt.xlabel(r's position [m]', fontsize=14)
    plt.ylabel(r'# lost electron', fontsize=14)


    # plt.xlim(0, spos[index][-1])
    # plt.plot(spos[np.intp(index)], nt_lost)
    plt.show()
    
    return c, inc[c],index, nt_lost


In [ ]:
# se eu quisesse mostrar a partir de qual valor de posição x começa a ter perda de eletrons na simulação de tracking
# como eu poderia fazer isso ?

# eu real, preciso fazer isso 

In [3]:
fitm = pymodels.si.create_accelerator()
fitm = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fitm)
eqparams = pyaccel.optics.beam_envelope.EqParamsFromBeamEnvelope(fitm)

# parameters to generate the beam
h_emitt = eqparams.emit1
v_emitt = h_emitt/100
sigmae = eqparams.espread0
bun_len = eqparams.bunlen

# defining optcis
twiss,*_ = pyop.calc_twiss(fitm, indices='closed')

# defining the index 
dicty = pyaccel.lattice.find_dict(fitm, 'fam_name')
nlk_index = dicty['InjNLKckr'][0] + 1
spos = pyaccel.lattice.find_spos(fitm, indices='closed')

# generating gaussian beam.
bunch = pyaccel.tracking.generate_bunch(50, envelope=None, emit1=h_emitt,
                                emit2=v_emitt, sigmae= sigmae, sigmas=bun_len, optics=twiss[nlk_index])


In [5]:
part_out = pyaccel.tracking.ring_pass(fitm, bunch[:,0:20], 7, True,nlk_index, True)

In [9]:
variable = varying_incmnts(fitm, bunch, 0, 50, nlk_index, 7)

In [99]:
spos = pyaccel.lattice.find_spos(fitm, indices='closed')

cont, lins, index, ntlost = plot_tracked(fitm, variable, 48, spos, 0.01)
ntlost

array([2., 2., 2., 2., 2., 2., 2., 3., 3., 2., 2., 3., 2., 3., 3., 2., 2.,
       3., 2., 2., 3., 2., 2., 3., 2., 3., 3., 2., 2., 3., 2., 3., 2., 2.,
       3., 2., 2., 2., 3., 3., 2., 2., 3., 3., 3., 3., 2., 3., 2., 2.])

In [61]:
lins

0.00875

In [46]:
ntlost

array([3., 5., 2., 3., 5., 5., 3., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       4., 5., 5., 4., 5., 5., 5., 5., 5., 6., 5., 5., 6., 6., 5., 6., 4.,
       5., 5., 6., 6., 5., 6., 3., 5., 3., 5.])

In [ ]:
# # anotações que eu pensei que pudessem ser importantes

# # definindo também o modelo fitado para calcular a distribuição de equilíbrio do feixe
# # eu não sei exatamente se essa celula no meu código é util, porque em um primeiro momento vamos variar apenas
# # as posições x, y, x' e y'

# fit_acc = pymodels.si.create_accelerator()
# fit_acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(fit_acc)
# tousan = tousclass.Tous_analysis(fit_acc)
# deltas = tousan.deltas
# n_turns = tousan.nturns

# orb = pyaccel.tracking.find_orbit6(acc, indices=[0, nlk_index])
# orb = orb[:, 1]
# np.zeros((6, deltas.size)), orb[:, None]

# rin = np.zeros((6, deltas.size))
# rin += orb[:,None]

# mean = np.zeros(6)
# env = pyop.beam_envelope.calc_beamenvelope(fit_acc)
# cov = env[nlk_index]
# particles = np.random.multivariate_normal(mean, cov, size=1).T

# # célula colocada aqui simplesmente para teste de verificação
# # ao executar o código abaixo é possível observar a distribuição gaussiana das posições em x
# plt.figure()
# plt.hist(particles[0], bins=200)
# plt.show()

# # esse procedimento é feito para ir variando os parâmetros de posição 
# vecy = np.linspace(4.8332669321064365e-09, 4.8332669321064365e-02, 1000)
# vecx = np.linspace(2.25702434e-08,2.25702434e-02, 1000)
# # com a função vstack do numpy podemos juntar dois arrays e formar uma matriz
# np.vstack((vecx, vecy))